# Lesson 2: Linear models with CNN features

## Linear model in Keras

* Dense() layers are just linear models, followed by a simple "activation function".
* Example linear model:

In [1]:
import os
import numpy as np

from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD, RMSprop
from keras.preprocessing.image import ImageDataGenerator

from sklearn.preprocessing import OneHotEncoder

import utils; reload(utils)
from utils import plots, get_batches

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
x = np.random.random((60, 2))
y = np.dot(x, [3., 7.]) + 1

In [3]:
x[:5]

array([[ 0.4829,  0.2328],
       [ 0.6003,  0.2383],
       [ 0.3625,  0.5428],
       [ 0.2129,  0.0321],
       [ 0.6797,  0.8611]])

In [4]:
y[:5]

array([ 4.0785,  4.4688,  5.8869,  1.8633,  9.0669])

Can create a simple linear model (Dense() - with no activation) and optimise using stochastic gradient descent, minimising mean squared error (mse):

In [5]:
lm = Sequential([Dense(1, input_shape=(2,))])
lm.compile(optimizer=SGD(lr=0.1), loss='mse')

In [6]:
lm.evaluate(x, y, verbose=0)

35.347666931152347

In [7]:
lm.fit(x, y, nb_epoch=5, batch_size=1)

Epoch 1/5
60/60 [==============================] - 0s - loss: 2.7263     
Epoch 2/5
60/60 [==============================] - 0s - loss: 0.2343     
Epoch 3/5
60/60 [==============================] - 0s - loss: 0.0763     
Epoch 4/5
60/60 [==============================] - 0s - loss: 0.0209     
Epoch 5/5
60/60 [==============================] - 0s - loss: 0.0055     


In [8]:
lm.evaluate(x, y, verbose=0)

0.0032060703417907161

In [9]:
# Note that the weighs are close to the y weights (3, 7) with an intercept of 1.
lm.get_weights()

[array([[ 2.8444],
        [ 6.9217]], dtype=float32), array([ 1.1402], dtype=float32)]

Can use a Dense() layer to convert the 1000 ImageNet predictions into probably of Dog vs Cat, by training a linear model to take 1000 predictions as input and return Dog or Cat as output.

In [10]:
path = "/home/ubuntu/nbs/data/male_female_training_set_20170610_cleaned/"
model_path = path + 'models'
if not os.path.exists(model_path): os.mkdir(model_path)

In [11]:
batch_size = 100

In [12]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

Approach:

1. Get true labels for every image.
2. Get 1,000 ImageNet category predictions for each image.
3. Feed predictions as input to simple linear model.

In [13]:
val_batches = get_batches(path + 'test', shuffle=False, batch_size=1)
batches = get_batches(path + 'train', shuffle=False, batch_size=1)

Found 2640 images belonging to 3 classes.
Found 17575 images belonging to 3 classes.


Can save the preprocessed arrays using bcolz, which also compresses the arrays.

In [14]:
import bcolz

def save_array(fname, arr):
    c = bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
    
def load_array(fname):
    return bcolz.open(fname)[:]

In [15]:
def get_data(path, target_size=(224, 224)):
    batches = get_batches(path, shuffle=False, batch_size=1, class_mode=None, target_size=target_size)
    return np.concatenate([batches.next() for i in range(batches.nb_sample)])

In [16]:
# Don't rerun these lines
val_data = get_data(path + 'test')

Found 2640 images belonging to 3 classes.


/home/ubuntu/anaconda2/lib/python2.7/site-packages/PIL/Image.py:869: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


In [17]:
trn_data = get_data(path + 'train')

Found 17575 images belonging to 3 classes.


In [18]:
trn_data.shape

(17575, 3, 224, 224)

In [19]:
save_array(model_path + 'train_data.bc', trn_data)
save_array(model_path + 'valid_data.bc', val_data)

# End lines

Can now load the training and validation data without having to recalculate.

In [15]:
trn_data = load_array(model_path + 'train_data.bc')
val_data = load_array(model_path + 'valid_data.bc')

Need to convert classes to one hot encoding, since Keras returns them as single columns.

In [16]:
val_batches.classes[:5]

array([0, 0, 0, 0, 0], dtype=int32)

In [17]:
def onehot(x):
    return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

val_labels = onehot(val_batches.classes)
trn_labels = onehot(batches.classes)

In [18]:
val_labels[:2]

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.]])

Grab predictions as 1000 potential classes from VGG.

In [19]:
trn_features = model.predict(trn_data, batch_size=batch_size)
val_features = model.predict(val_data, batch_size=batch_size)

In [20]:
trn_features.shape

(17575, 1000)

In [21]:
save_array(model_path + 'train_lastlayer_features.bc', trn_features)
save_array(model_path + 'valid_lastlayer_features.bc', val_features)

In [22]:
trn_features = load_array(model_path + 'train_lastlayer_features.bc')
val_features = load_array(model_path + 'valid_lastlayer_features.bc')

Now to define a simple linear model that takes 1000 labels as inputs and outputs 3 categories (female, male, none).

In [23]:
# 1000 inputs, since that's the saved features, and 3 outputs.
lm = Sequential([ Dense(3, activation='softmax', input_shape=(1000,)) ])
lm.compile(optimizer=RMSprop(lr=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
batch_size = 4

In [25]:
lm.fit(trn_features, trn_labels, nb_epoch=3, batch_size=batch_size, 
       validation_data=(val_features, val_labels))

Train on 17575 samples, validate on 2640 samples
Epoch 1/3
17575/17575 [==============================] - 6s - loss: 0.9007 - acc: 0.7536 - val_loss: 0.9533 - val_acc: 0.7602
Epoch 2/3
17575/17575 [==============================] - 6s - loss: 1.0619 - acc: 0.7593 - val_loss: 1.1063 - val_acc: 0.7530
Epoch 3/3
17575/17575 [==============================] - 6s - loss: 1.1657 - acc: 0.7533 - val_loss: 1.3005 - val_acc: 0.7330


In [26]:
lm.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_5 (Dense)                  (None, 3)             3003        dense_input_2[0][0]              
Total params: 3003
____________________________________________________________________________________________________


### About activation functions

* Adding an activation parameter to a layer means another function is run on the layer after it's calculated.
* Nearly all deep model layers have an activation function that's non-linear like ``tanh``, ``sigmoid (1 / (1 + exp(x)))`` or ``relu (max(0,x)``.
  * Because if you stack linear layers, you end up with a linear layer. Eg ```(2 * x) * (-2 * x) = -4*x```.
  * But, by adding an activation function, you could get: ```-2 * max(0, 2 * x)```, which does not simplify.
  * This let's you create arbitrarily complex functions.
* The last layer should have a different activation function to the other layers, to ensure the output is the appropriate form.
  * Since our data is one-hot encoded, we want to ensure a single activation, is way higher than the rest, so softmax is used.
  * Softmax is defined as ```exp(x[i]) / sum(exp(x))```. As below:
  
  ```
  output	exp	softmax
   4.51	    91.05	0.99
  -2.11	    0.12	0.00
  -0.57	    0.57	0.01
  -4.06	    0.02	0.00
  -3.85	    0.02	0.00
            91.78   1.00
  ```

## Modifying the model

### Retrain the last layer's linear model.

* In the original VGG16 network, the last layer is Dense (linear model). In the example above, we are simply adding a linear layer on top of it, which is inefficient.
* What we want to do instead is replace the final layer with one specific to our problem.
* To do it, you remove the last layer from the model and fix all the others.

In [27]:
vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [28]:
# Remove the last Dense layers.
model.pop()

# Set the other layers to not be trainable.
for layer in model.layers:
    layer.trainable = False

In [29]:
model.add(Dense(3, activation='softmax'))

Now compile the model and retrain.

In [30]:
gen = ImageDataGenerator()
batches = gen.flow(trn_data, trn_labels, batch_size=batch_size, shuffle=True)
val_batches = gen.flow(val_data, val_labels, batch_size=batch_size, shuffle=False)

In [31]:
opt = RMSprop(lr=0.1)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=1, 
                    validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
17575/17575 [==============================] - 528s - loss: 6.2318 - acc: 0.6116 - val_loss: 5.1691 - val_acc: 0.6780


In [33]:
model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=1, 
                    validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
17575/17575 [==============================] - 528s - loss: 6.0655 - acc: 0.6230 - val_loss: 4.9458 - val_acc: 0.6924


## Retraining more layers

* As well as fine tuning the last dense layer, you can retrain the other dense layers too.

### Intro to back-propagation

* Back propagation is another word for the chain rule.
* Chain rule is how you calculate the gradient of a function of a function.
  * Given ``f(u)`` where ``u`` is a function: ``g(x)``. The gradient would be ``f(u) * g(x)``
* Can calculate the derivative at any layer by multiplying the gradients at that layer and all the following layers together.

### Training multiple layers

* Set all Dense layers to trainable:

In [34]:
layers = model.layers
# Get the index of the first dense layer...
first_dense_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense][0]
# ...and set this and all subsequent layers to trainable
for layer in layers[first_dense_idx:]: layer.trainable=True

Since the architecture is the same, don't need to compile the model.

In [36]:
from keras import backend as K

K.set_value(opt.lr, 0.01)
model.fit_generator(
    batches, samples_per_epoch=batches.N, nb_epoch=3, 
    validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/3
17575/17575 [==============================] - 526s - loss: 5.9898 - acc: 0.6275 - val_loss: 5.1130 - val_acc: 0.6822
Epoch 2/3
17575/17575 [==============================] - 527s - loss: 5.9997 - acc: 0.6273 - val_loss: 5.2451 - val_acc: 0.6731
Epoch 3/3
17575/17575 [==============================] - 527s - loss: 5.9660 - acc: 0.6292 - val_loss: 5.2246 - val_acc: 0.6746


In [37]:
model.save_weights(model_path + 'gender_3_dense_retrain.h5')